In [67]:
#%%
import pandas as pd

def readVaricelle():
    filename = 'Data/cas_varicelles.csv'
    df = pd.read_csv(filename)
    #Select only Ile de France
    df = df[df['geo_insee']==11]
    df['week'] = df['week'].astype(str)
    print(type(df['week']))
    df['year'] = df['week'].str[0:4].astype(int)
    df['week'] = df['week'].str[4:6].astype(int)
    return df

def readWeather():
    filename = 'Data/Arpajon_weather.csv'
    df = pd.read_csv(filename)
    df['date_time']= pd.to_datetime(df['date_time'])
    df['year'] = df['date_time'].dt.isocalendar().year
    df['week'] = df['date_time'].dt.isocalendar().week
    return df

def readDiarrhee():
    filename = 'Data/diarrhee_aigues.csv'
    df = pd.read_csv(filename)
    #Select only Ile de France
    df = df[df['geo_insee']==11]
    df['week'] = df['week'].astype(str)
    df['year'] = df['week'].str[0:4].astype(int)
    df['week'] = df['week'].str[4:6].astype(int)
    return df

def readInterventions():
    filename = 'Data/interventions-hebdo-2010-2017.csv'
    df = pd.read_csv(filename, sep=";")
    # remove info on type of intervention and city
    df = df.drop(columns=['ope_code_insee','ope_categorie','ope_code_postal','ope_nom_commune'], axis=1)
    # group and count number of operation by week
    df = df.groupby(['ope_annee','ope_semaine'], as_index=False).sum()
    df = df.rename(columns={"ope_annee": "year", "ope_semaine": "week"})
    return df

def readVacances():
    filename = 'Data/vacances.csv'
    df = pd.read_csv(filename)
    #remove zone a, zone c and nom
    df = df.drop(columns=['vacances_zone_a','vacances_zone_b','nom_vacances'], axis=1)
    df['date']= pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.isocalendar().year
    df['week'] = df['date'].dt.isocalendar().week
    return df

def readFeries():
    filename = 'Data/jours_feries_metropole.csv'
    df = pd.read_csv(filename)
    df = df.drop(columns=['annee','zone','nom_jour_ferie'], axis=1)
    df['ferie'] = True
    df['date']= pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.isocalendar().year
    df['week'] = df['date'].dt.isocalendar().week
    return df

def readGrippes():
    filename = 'Data/symptomes_grippaux.csv'
    df = pd.read_csv(filename)
    #Select only Ile de France
    df = df[df['geo_insee']==11]
    # make string version of original column, call it 'col'
    df['week'] = df['week'].astype(str)
    df['year'] = df['week'].str[0:4].astype(int)
    df['week'] = df['week'].str[4:6].astype(int)
    return df

def readVacancesFeries():
    df1 = readVacances()
    df2 = readFeries()
    #Create new dataframe with merged vacances and feries
    df = df1.merge(df2, how='outer')
    df['ferie'].fillna(False, inplace=True)
    df = df.drop(columns=['date'], axis=1)
    df = df.groupby(['year','week'], as_index=False).sum()
    return df

def mergeAll(dfIntervention, dfVaricelle, dfWeather, dfDiarrhee, dfGrippe, dfVacFerie):
    df = pd.merge(dfIntervention, dfVaricelle, how='left', on=['year', 'week'])
    df = pd.merge(df, dfWeather, how='left', on=['year', 'week'])
    df = pd.merge(df, dfDiarrhee, how='left', on=['year', 'week'])
    df = pd.merge(df, dfGrippe, how='left', on=['year', 'week'])
    df = pd.merge(df, dfVacFerie, how='left', on=['year', 'week'])
    return df

dfI = readInterventions()
dfVF = readVacancesFeries()
dfV = readVaricelle()
dfG = readGrippes()
dfD = readDiarrhee()
dfW = readWeather()
"""
print(readVacancesFeries())"""
print(readVaricelle())
"""
print(readFeries())
print(readDiarrhee())
print(readWeather())"""
print(readInterventions())
"""
#print(readVacances())
print(readGrippes())
#print(readFeries())"""

df = mergeAll(dfI,dfV,dfW,dfD,dfG,dfVF)
df.to_csv('Data/data_merged.csv', index=False)  
print(df)
# %%

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
      week  indicator   inc  inc_low  inc_up  inc100  inc100_low  inc100_up  \
7       52          7   368        0     920       3           0          8   
20      51          7   799        0    1962       7           0         17   
33      50          7  2228      476    3980      18           4         32   
46      49          7   800        0    1679       7           0         14   
59      48          7  1029        0    2145       8           0         17   
...    ...        ...   ...      ...     ...     ...         ...        ...   
5363     5          7  6835     2376   11294      59          21         97   
5376     4          7  1284       89    2479      11           1         21   
5389     3          7  1420        0    3082      12           0         26   
5402     2          7  3120      267    5973      27           3         51   
5415     1          7  3735      242    7228      32       

In [65]:
readVacancesFeries()

,date,vacances_zone_c,year,week,ferie
0,2010-01-01,True,2009,53,True
1,2010-01-02,True,2009,53,False
2,2010-01-03,True,2009,53,False
3,2010-01-04,False,2010,1,False
4,2010-01-05,False,2010,1,False
...,...,...,...,...,...
2917,2017-12-27,True,2017,52,False
2918,2017-12-28,True,2017,52,False
2919,2017-12-29,True,2017,52,False
2920,2017-12-30,True,2017,52,False


,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
7,201752,7,368,0,920,3,0,8,11,ILE-DE-FRANCE
20,201751,7,799,0,1962,7,0,17,11,ILE-DE-FRANCE
33,201750,7,2228,476,3980,18,4,32,11,ILE-DE-FRANCE
46,201749,7,800,0,1679,7,0,14,11,ILE-DE-FRANCE
59,201748,7,1029,0,2145,8,0,17,11,ILE-DE-FRANCE
...,...,...,...,...,...,...,...,...,...,...
5363,201005,7,6835,2376,11294,59,21,97,11,ILE-DE-FRANCE
5376,201004,7,1284,89,2479,11,1,21,11,ILE-DE-FRANCE
5389,201003,7,1420,0,3082,12,0,26,11,ILE-DE-FRANCE
5402,201002,7,3120,267,5973,27,3,51,11,ILE-DE-FRANCE
